In [2]:
import os 
import numpy as np 
import pandas as pd 

### handle data from Nansen NFT Paradise 
- I don't have the csv access, so I copy-pasted frm the website and rehandled the data 
- already filtered (floor price between 0.05 ~ 0.12)
- from https://pro.nansen.ai/nft-paradise


In [5]:
df_data[df_data["NFT_collections"] == "Rubber Duck Bath Party"]

,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed
22,Rubber Duck Bath Party,8.47,974,0.0974,0.085,-0.25,-0.34,5929,24d ago


In [6]:
# load data 
data_path = os.path.join(os.getcwd(), "data")
file_name = os.path.join(data_path, "nansen_nft_paradise_1.xlsx")
sheet_to_df_map = pd.read_excel(file_name, sheet_name=None, header=None)

# reshape data 
col_names = ["NFT_collections", "Volumne_ETH", "Mkt_Cap_ETH", "Avg_Price_ETH", 
             "Floor_ETH", "Floor_24H", "Floor_3D", "#Wallets", "Deployed"]
lst_data = []
for sheet in sheet_to_df_map:
    df_new = pd.DataFrame(data = sheet_to_df_map[sheet].values.reshape(-1, 9), columns=col_names)
    lst_data.append(df_new)

# concat and save data 
df_data = pd.concat(lst_data)
df_data.reset_index(inplace=True, drop=True)
df_data = df_data.loc[df_data["NFT_collections"] != "NFT Collection"]
df_data.drop_duplicates(subset= "NFT_collections", inplace=True)
df_data.to_excel(os.path.join(data_path, "nansen_nft_paradise_handled.xlsx"))
df_data.head(), df_data.shape

(  NFT_collections Volumne_ETH Mkt_Cap_ETH Avg_Price_ETH Floor_ETH Floor_24H  \
 0     TrippyToadz         426         154        0.0735     0.068         0   
 1  Juicebox Frens         211         950        0.1363    0.0859      -0.2   
 2     Mini Supers         149         139        0.0728     0.084         -   
 3    MPL Official          75        2299        0.2299      0.11      -0.3   
 4       ToonSquad          65         487        0.0809    0.0558   -0.0084   
 
   Floor_3D #Wallets Deployed  
 0        -     2496  40h ago  
 1     0.24     2890   5d ago  
 2        -     1650  37h ago  
 3    -0.46     4033   6d ago  
 4     -0.3     2454   5d ago  ,
 (152, 9))

### Twitter hotness filtration 

cretiria (can be customized)
- mean_tweets_per_week
- mean_tweets_per_week_exclu_retweets
- mean_like_count
- mean_retweet_count
- mean_reply_count

In [7]:
import datetime, tqdm 
from twitter_giveaway_bot import *

# connect to twitter api v2 
client = connect()

In [18]:
# Twitter hotness filtration 
def process_project_from_tweeter(client, project_name):

    # print("-*80")
    # print(f"project_name: {project_name}")
    # init dataframe to store value 
    df_data = pd.DataFrame(columns=["account", "NFT_collections", "followers_count", "following_count", "tweet_count", 
                                    "mean_tweets_per_week", "mean_tweets_per_week_exclu_retweets", "mean_like_count", "mean_retweet_count", "mean_reply_count", "twitter_url"])
    df_data.set_index("account", inplace=True)

    # search related recent tweets 
    recent_tweets =  client.search_recent_tweets(project_name, user_auth=True, max_results=20)
    if recent_tweets.data is not None:
        # find all twitter accounts in the recent tweets  
        lst = [re.findall(r"@([a-zA-Z0-9_]+)", tweet["text"]) for tweet in recent_tweets.data]
        potential_twitter_accounts = list(set([item for sublist in lst for item in sublist]))
        # print(f"potential_twitter_accounts: {potential_twitter_accounts}")
        # find the true one 
        # assume that the twitter account should at least contains one part of the project name  
        lst2 = [account.lower() for account in potential_twitter_accounts]
        project_name_lower =  project_name.lower().split(" ")
        true_potential_twitter_accounts = set([potential_twitter_accounts[i] for i in range(len(lst2)) for j in project_name_lower if j in lst2[i]])
        # print(f"true_potential_twitter_accounts: {true_potential_twitter_accounts}")
        
        # if cant find the twitter accounts and the set is empty, we escape
        if true_potential_twitter_accounts:
            for account in true_potential_twitter_accounts:
                # get account info 
                user = client.get_user(username=account, user_fields=["public_metrics"], user_auth=True).data
                if user:
                    account_info = user["public_metrics"]
                    followers_count = account_info["followers_count"]
                    following_count = account_info["following_count"]
                    tweet_count = account_info["tweet_count"]
                    
                    # get account recent tweets 
                    latest_tweets = client.get_users_tweets(
                        user["id"],
                        user_auth=True,
                        max_results=100,
                        tweet_fields=["text", "public_metrics"],
                        start_time = datetime.datetime.now() - datetime.timedelta(days=14)
                    ).data

                    if latest_tweets:
                        # statistics 
                        # !! for average tweets per week, since we can only access 100 recent tweets, if the number of recent tweets
                        # is larger than 100, this measure is underestimated  
                        mean_tweets_per_week = len(latest_tweets) / 14 * 7
                        total_number_of_tweets_exclu_retweets = sum([1 if "RT" not in one_tweet["text"] else 0 for one_tweet in latest_tweets])
                        mean_tweets_per_week_exclu_retweets = total_number_of_tweets_exclu_retweets  / 14 * 7
                        mean_like_count = np.mean([one_tweet["public_metrics"]["like_count"] for one_tweet in latest_tweets])
                        mean_retweet_count = np.mean([one_tweet["public_metrics"]["retweet_count"] for one_tweet in latest_tweets])
                        mean_reply_count = np.mean([one_tweet["public_metrics"]["reply_count"] for one_tweet in latest_tweets])

                        # 
                        twitter_url = f"https://twitter.com/{account}"
                        df_data.loc[account, :] = [project_name, followers_count, following_count, tweet_count, 
                                                mean_tweets_per_week, mean_tweets_per_week_exclu_retweets, mean_like_count, mean_retweet_count, mean_reply_count,
                                                twitter_url]

    # else:
    #     print(f"{project_name} dont have recent tweets")
    
    return df_data


In [19]:
# test 
project_name = "TrippyToadz"
process_project_from_tweeter(client, project_name)

# df = process_project_from_tweeter(client, "Prime Kong Planet")
# df.to_excel("Kong.xlsx")

,NFT_collections,followers_count,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url
account,,,,,,,,,,
trippytoadznft,TrippyToadz,23129,1192,977,50.0,35.5,115.33,110.78,32.15,https://twitter.com/trippytoadznft


In [20]:
lst_df_tweets_stats = []
for project_name in tqdm.tqdm(df_data["NFT_collections"][:]):
    try:
        df_tweets_stats = process_project_from_tweeter(client, project_name)
        lst_df_tweets_stats.append(df_tweets_stats)
    except Exception:
        # twitter has request limits for API 
        # after 15 minites 
        print("Need to relax for 16 minites")
        time.sleep(16 * 60 * 60)
    time.sleep(0.01)
df_tweets_stats_all = pd.concat(lst_df_tweets_stats)
df_tweets_stats_all.reset_index(inplace=True)
df_tweets_stats_all.drop_duplicates(subset="account", inplace=True)

100%|██████████| 152/152 [02:22<00:00,  1.07it/s]


In [21]:
# merge data and statistics 
df_results = pd.merge(df_data, df_tweets_stats_all, how="inner")
df_results.to_excel(os.path.join(os.getcwd(), "results", "results_nft_project_original.xlsx"))

df_results

,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed,account,followers_count,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url
0,TrippyToadz,426,154,0.0735,0.068,0,-,2496,40h ago,trippytoadznft,23129,1192,977,50.0,35.5,115.33,110.78,32.15,https://twitter.com/trippytoadznft
1,Juicebox Frens,211,950,0.1363,0.0859,-0.2,0.24,2890,5d ago,Juiceboxfrens,6561,1462,1017,50.0,43.0,20.68,10.79,5.37,https://twitter.com/Juiceboxfrens
2,Mini Supers,149,139,0.0728,0.084,-,-,1650,37h ago,minisupersnft,11536,2,199,50.0,26.0,18.06,27.09,2.01,https://twitter.com/minisupersnft
3,MPL Official,75,2299,0.2299,0.11,-0.3,-0.46,4033,6d ago,mpl,66,150,547,0.5,0.0,0.0,2905.0,0.0,https://twitter.com/mpl
4,ToonSquad,65,487,0.0809,0.0558,-0.0084,-0.3,2454,5d ago,ToonSquadNFT,51262,48,839,50.0,14.0,13.99,13.67,1.25,https://twitter.com/ToonSquadNFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Seabums,1.62,222,0.038,0.03,0.0843,0.5,3198,109d ago,SeabumsNFTs,5232,430,427,28.0,18.5,10.267857,6.035714,2.803571,https://twitter.com/SeabumsNFTs
153,Guardians of the Metaverse,1.61,804,0.0804,0.06,0.0942,0.41,4161,198d ago,WhisperOfArt,1108,870,577,49.5,37.5,5.10101,30.676768,1.111111,https://twitter.com/WhisperOfArt
154,Guardians of the Metaverse,1.61,804,0.0804,0.06,0.0942,0.41,4161,198d ago,metaguardians,24246,242,1845,22.5,15.5,30.466667,19.555556,7.133333,https://twitter.com/metaguardians
155,Guardians of the Metaverse,1.61,804,0.0804,0.06,0.0942,0.41,4161,198d ago,TheBinanceNFT,640724,317,1913,50.0,41.5,106.36,56.14,43.2,https://twitter.com/TheBinanceNFT


In [22]:
# filtration 
con_followers_count = df_results.loc[:, "followers_count"] > 40000
con_mean_tweets_per_week_exclu_retweets = df_results.loc[:, "mean_tweets_per_week_exclu_retweets"] > 2
con_mean_like_count = df_results.loc[:, "mean_like_count"] > 10
con_mean_retweet_count = df_results.loc[:, "mean_retweet_count"] > 10
con_mean_reply_count= df_results.loc[:, "mean_reply_count"] > 10
print(f"% con_followers_count: {np.mean(con_followers_count)}")
print(f"% con_mean_tweets_per_week_exclu_retweets: {np.mean(con_mean_tweets_per_week_exclu_retweets)}")
print(f"% con_mean_like_count: {np.mean(con_mean_like_count)}")
print(f"% con_mean_retweet_count: {np.mean(con_mean_retweet_count)}")
print(f"% con_mean_reply_count: {np.mean(con_mean_reply_count)}")

# combind condition 
con_all = con_followers_count & con_mean_tweets_per_week_exclu_retweets & con_mean_like_count & con_mean_retweet_count & con_mean_reply_count
df_results_filtered = df_results.loc[con_all, :]
print(f"total {df_results.count()[0]} NFT projects within floor price range [0.05 ~ 0.12]")
print(f"there are {df_results_filtered.count()[0]} satisfying the twitter condition")
df_results_filtered.to_excel(os.path.join(os.getcwd(), "results", "results_nft_project_filtered.xlsx"))
df_results_filtered

% con_followers_count: 0.2229299363057325
% con_mean_tweets_per_week_exclu_retweets: 0.910828025477707
% con_mean_like_count: 0.6369426751592356
% con_mean_retweet_count: 0.6815286624203821
% con_mean_reply_count: 0.35668789808917195
total 157 NFT projects within floor price range [0.05 ~ 0.12]
there are 23 satisfying the twitter condition


,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed,account,followers_count,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url
6,Villagers of XOLO,50,3274,0.2669,0.11,-0.2,-0.44,4858,10d ago,PlanetXOLO,45647,2,49,12.0,10.0,257.875,81.125,15.833333,https://twitter.com/PlanetXOLO
11,Super Ordinary Villains - SOV,14,797,0.0897,0.07,-0.23,-0.41,4919,14d ago,SOVillains,77249,9,196,26.0,9.0,165.826923,170.461538,43.096154,https://twitter.com/SOVillains
16,Squishiverse,11,1335,0.1502,0.1,-0.2,0.0903,1960,40d ago,SquishiverseNFT,61423,79,322,50.0,30.0,78.23,93.04,51.62,https://twitter.com/SquishiverseNFT
18,Wabi Sabi Collective,8.42,673,0.1186,0.105,-0.13,0.28,1919,20d ago,WabiSabiNFT,113397,17,419,34.5,24.0,1288.15942,1271.57971,1062.73913,https://twitter.com/WabiSabiNFT
23,Crypto Bears,8.03,591,0.0887,0.079,1.03,0.58,3796,39d ago,crypto_birb,653554,4118,55617,50.0,33.5,63.34,14.15,16.77,https://twitter.com/crypto_birb
24,Crypto Bears,8.03,591,0.0887,0.079,1.03,0.58,3796,39d ago,Bitboy_Crypto,825133,645,23642,50.0,42.5,416.89,50.66,118.12,https://twitter.com/Bitboy_Crypto
27,Wulfz,7.98,1082,0.1376,0.093,0.45,0.35,991,85d ago,wulfznft,87642,13,451,30.0,18.0,700.05,689.683333,658.3,https://twitter.com/wulfznft
32,The Sunnies,7.24,720,0.0787,0.066,-0.14,-0.27,4678,24d ago,thesunniesnft,41646,27,377,29.5,15.0,112.033898,122.881356,13.050847,https://twitter.com/thesunniesnft
47,C86 Cyborg,4.85,589,0.0685,0.074,1.2,0.87,3902,78d ago,Cyborg86nft,62865,20,235,23.5,18.0,47.212766,49.638298,14.106383,https://twitter.com/Cyborg86nft
49,FishyFam,4.38,1326,0.1326,0.118,-0.1,-0.12,5966,58d ago,fishyfamNFT,75955,513,433,19.5,16.0,458.128205,264.410256,190.564103,https://twitter.com/fishyfamNFT


### Find discord urls 



In [23]:
import re
import requests
from bs4 import BeautifulSoup

# send request and return soup
def quickSoup(url):
    try:
        header = {}
        header['User-Agent'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
        soup = BeautifulSoup(requests.get(url, headers=header, timeout=10).content, 'html.parser')
        return soup
    except Exception:
        return None
    
def find_discord_url(username):
   
    print("-"*80)
    print(f"username: {username}")
    discord_url = None
    
    ##
    # find discord url from twitter
    # 1. discription
    # 2. linktree 
    user = client.get_user(username=username, user_fields=[ "entities"]).data
    entities = user["entities"]
    for i in entities:
        if "urls" in entities[i]:
            urls = entities[i]["urls"]
            for j in urls:
                expanded_url = j["expanded_url"]
                if any(w in expanded_url for w in ["Discord", "discord"]):
                    discord_url = expanded_url
                if "linktr" in expanded_url:
                    print("through linktr")
                    # print(expanded_url)
                    # request the linktr and find the discord url
                    soup = quickSoup(expanded_url)
                    for a in soup.find_all('a', href=True):
                        if any(w in a["href"] for w in ["Discord", "discord"]):
                            # print("Found the discord URL:", a['href'])
                            discord_url = a['href']
    
    return discord_url
                
    # user_description = user["description"]
    # # find discord url from linktree 
    


    
    # if "Discord" in user_description:
    #     discord_url = user_description.split("Discord: ")[-1]
    #     print(f"discord_url: {discord_url}")
    # else:
    #     print("no discord in description")
    #     print(f"user_description: {user_description}")      

    # print(f"discord_url: {discord_url}")
    
    
def find_number_of_memeber_in_discord(discord_url):
        # scrape discord info 
        soup = quickSoup(discord_url)
        content = ''.join([str(i) for i in soup.contents])
        phrase = '_'.join(re.findall(r"[0-9A-Za-z, ]+ member[s].", content))
        n_members = re.findall(r"[0-9,]+", phrase)[0].replace(",","").replace(" member", "")
        print(n_members)
    
        return n_members



In [24]:
df_results_filtered.loc[:, "n_discord"] = 0
df_results_filtered.loc[:, "url_d"] = ""
# get number of memebers in discord:
for i in range(len(df_results_filtered.index)):
    discord_url = find_discord_url(df_results_filtered.account.iloc[i])
    print(f"discord_url: {discord_url}")
    if discord_url is not None:
        n_members = find_number_of_memeber_in_discord(discord_url)
        df_results_filtered["n_discord"].iloc[i] = n_members
        df_results_filtered["url_d"].iloc[i] = str(discord_url)


C:\Users\wangj\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


--------------------------------------------------------------------------------
username: PlanetXOLO
14049
--------------------------------------------------------------------------------
username: SOVillains


C:\Users\wangj\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


21661
--------------------------------------------------------------------------------
username: SquishiverseNFT
through linktr


C:\Users\wangj\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()


12374
--------------------------------------------------------------------------------
username: WabiSabiNFT
22330
--------------------------------------------------------------------------------
username: crypto_birb
--------------------------------------------------------------------------------
username: Bitboy_Crypto
--------------------------------------------------------------------------------
username: wulfznft
through linktr
22004
--------------------------------------------------------------------------------
username: thesunniesnft
1838
--------------------------------------------------------------------------------
username: Cyborg86nft
through linktr
302052
--------------------------------------------------------------------------------
username: fishyfamNFT
74713
--------------------------------------------------------------------------------
username: CryptoHipposNFT
22296
--------------------------------------------------------------------------------
username: veve_off

In [48]:
df_results_filtered = df_results_filtered.astype({"n_discord":float})
df_results_filtered2 = df_results_filtered.loc[df_results_filtered["n_discord"]> 15000, :]
df_results_filtered2.to_excel(os.path.join(os.getcwd(), "results", "results_nft_project_filtered.xlsx"))
df_results_filtered2

,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed,account,...,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url,n_discord,url_d
11,Super Ordinary Villains - SOV,14,797,0.0897,0.07,-0.23,-0.41,4919,14d ago,SOVillains,...,9,196,26.0,9.0,165.826923,170.461538,43.096154,https://twitter.com/SOVillains,21661.0,https://discord.gg/sovillains
18,Wabi Sabi Collective,8.42,673,0.1186,0.105,-0.13,0.28,1919,20d ago,WabiSabiNFT,...,17,419,34.5,24.0,1288.15942,1271.57971,1062.73913,https://twitter.com/WabiSabiNFT,22330.0,http://discord.gg/wabisabicollective
27,Wulfz,7.98,1082,0.1376,0.093,0.45,0.35,991,85d ago,wulfznft,...,13,451,30.0,18.0,700.05,689.683333,658.3,https://twitter.com/wulfznft,22004.0,http://discord.gg/wulfz
47,C86 Cyborg,4.85,589,0.0685,0.074,1.2,0.87,3902,78d ago,Cyborg86nft,...,20,235,23.5,18.0,47.212766,49.638298,14.106383,https://twitter.com/Cyborg86nft,302052.0,http://discord.com/invite/cyborg86
49,FishyFam,4.38,1326,0.1326,0.118,-0.1,-0.12,5966,58d ago,fishyfamNFT,...,513,433,19.5,16.0,458.128205,264.410256,190.564103,https://twitter.com/fishyfamNFT,74713.0,http://discord.gg/fishyfam
52,Crypto Hippos NFT,4.18,541,0.0755,0.048,-0.0303,-0.0233,2429,29d ago,CryptoHipposNFT,...,20,211,18.5,8.5,132.216216,122.486486,22.972973,https://twitter.com/CryptoHipposNFT,22296.0,https://discord.gg/cryptohippos
64,House Of Legends,3.59,1995,0.1996,0.0599,0.0328,0.2,2673,93d ago,veve_official,...,196,4393,44.5,42.0,410.494382,74.168539,68.359551,https://twitter.com/veve_official,116076.0,https://discord.gg/veve
86,Gooniez Gang,2.57,1200,0.135,0.085,-0.0957,-0.12,5734,41d ago,GooniezNFT,...,123,561,12.0,9.5,228.125,171.416667,140.791667,https://twitter.com/GooniezNFT,134584.0,https://discord.gg/gooniez
96,The Ramen Shop NFT,2.25,25,0.0487,0.05,0.24,-0.11,284,5d ago,SoulZ_NFT,...,10,361,14.0,10.5,618.964286,748.964286,537.357143,https://twitter.com/SoulZ_NFT,97157.0,https://discord.com/invite/soulz
126,Fortress-Arena NFT,1.93,1337,0.3593,0.05,-0.0909,-0.11,452,77d ago,cerealclubnft,...,21,328,36.0,19.5,226.444444,199.763889,163.555556,https://twitter.com/cerealclubnft,29962.0,http://discord.gg/cerealclub


,NFT_collections,Volumne_ETH,Mkt_Cap_ETH,Avg_Price_ETH,Floor_ETH,Floor_24H,Floor_3D,#Wallets,Deployed,account,...,following_count,tweet_count,mean_tweets_per_week,mean_tweets_per_week_exclu_retweets,mean_like_count,mean_retweet_count,mean_reply_count,twitter_url,n_discord,url_d
11,Super Ordinary Villains - SOV,14,797,0.0897,0.07,-0.23,-0.41,4919,14d ago,SOVillains,...,9,196,26.0,9.0,165.826923,170.461538,43.096154,https://twitter.com/SOVillains,21661,https://discord.gg/sovillains
18,Wabi Sabi Collective,8.42,673,0.1186,0.105,-0.13,0.28,1919,20d ago,WabiSabiNFT,...,17,419,34.5,24.0,1288.15942,1271.57971,1062.73913,https://twitter.com/WabiSabiNFT,22330,http://discord.gg/wabisabicollective
27,Wulfz,7.98,1082,0.1376,0.093,0.45,0.35,991,85d ago,wulfznft,...,13,451,30.0,18.0,700.05,689.683333,658.3,https://twitter.com/wulfznft,22004,http://discord.gg/wulfz
47,C86 Cyborg,4.85,589,0.0685,0.074,1.2,0.87,3902,78d ago,Cyborg86nft,...,20,235,23.5,18.0,47.212766,49.638298,14.106383,https://twitter.com/Cyborg86nft,302052,http://discord.com/invite/cyborg86
49,FishyFam,4.38,1326,0.1326,0.118,-0.1,-0.12,5966,58d ago,fishyfamNFT,...,513,433,19.5,16.0,458.128205,264.410256,190.564103,https://twitter.com/fishyfamNFT,74713,http://discord.gg/fishyfam
52,Crypto Hippos NFT,4.18,541,0.0755,0.048,-0.0303,-0.0233,2429,29d ago,CryptoHipposNFT,...,20,211,18.5,8.5,132.216216,122.486486,22.972973,https://twitter.com/CryptoHipposNFT,22296,https://discord.gg/cryptohippos
64,House Of Legends,3.59,1995,0.1996,0.0599,0.0328,0.2,2673,93d ago,veve_official,...,196,4393,44.5,42.0,410.494382,74.168539,68.359551,https://twitter.com/veve_official,116076,https://discord.gg/veve
86,Gooniez Gang,2.57,1200,0.135,0.085,-0.0957,-0.12,5734,41d ago,GooniezNFT,...,123,561,12.0,9.5,228.125,171.416667,140.791667,https://twitter.com/GooniezNFT,134584,https://discord.gg/gooniez
96,The Ramen Shop NFT,2.25,25,0.0487,0.05,0.24,-0.11,284,5d ago,SoulZ_NFT,...,10,361,14.0,10.5,618.964286,748.964286,537.357143,https://twitter.com/SoulZ_NFT,97157,https://discord.com/invite/soulz
126,Fortress-Arena NFT,1.93,1337,0.3593,0.05,-0.0909,-0.11,452,77d ago,cerealclubnft,...,21,328,36.0,19.5,226.444444,199.763889,163.555556,https://twitter.com/cerealclubnft,29962,http://discord.gg/cerealclub
